In [1]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.datasets import fetch_openml
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import log_loss
import tensorflow as tf
from tensorflow import keras

2025-04-29 10:29:23.541914: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1745911763.629829    7176 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1745911763.653855    7176 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-04-29 10:29:23.808496: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


# loading data and preprocessing

In [3]:
INPUT_SIZE = 784
HIDDEN_SIZE = 32
OUTPUT_SIZE = 10

from keras.datasets import mnist


(x_train,y_train),(x_test, y_test) = mnist.load_data()
print("images train before normalize and processing",x_train.shape)

x_train = tf.reshape(x_train, [-1, 28*28])
x_test = tf.reshape(x_test, [-1, 28*28])

print("images train after reshaping",x_train.shape)
# x_train[1]

# handling pixels to be only black and white from gray scale
x_train = tf.cast(x_train, tf.float32) / 255.0
x_test = tf.cast(x_test, tf.float32) / 255.0


y_train_hot = keras.utils.to_categorical(y_train, num_classes=10)
y_test_hot = keras.utils.to_categorical(y_test, num_classes=10)
print(y_train_hot.shape)
print(y_test_hot.shape)

y_train_hot = tf.convert_to_tensor(y_train_hot, dtype=tf.float32)
y_test_hot = tf.convert_to_tensor(y_test_hot, dtype=tf.float32)



images train before normalize and processing (60000, 28, 28)
images train after reshaping (60000, 784)
(60000, 10)
(10000, 10)


I0000 00:00:1745833014.153039   12211 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 1225 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 3050 Laptop GPU, pci bus id: 0000:01:00.0, compute capability: 8.6
2025-04-28 12:36:54.154724: W external/local_xla/xla/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 47040000 exceeds 10% of free system memory.


# neural netowrk structure

In [4]:
INPUT_SIZE = 784
HIDDEN_SIZE = 32
OUTPUT_SIZE = 10

def sigmoid(x):
    return tf.nn.sigmoid(x)

def softmax(x):
    return tf.nn.softmax(x)

w1_size = INPUT_SIZE * HIDDEN_SIZE
b1_size = HIDDEN_SIZE
w2_size = HIDDEN_SIZE * OUTPUT_SIZE
b2_size = OUTPUT_SIZE
D = w1_size + b1_size + w2_size + b2_size


def decode_weights(vector):
    
    w1_size = INPUT_SIZE * HIDDEN_SIZE
    b1_size = HIDDEN_SIZE
    w2_size = HIDDEN_SIZE * OUTPUT_SIZE
    b2_size = OUTPUT_SIZE
    
    idx = 0
    w1 = tf.reshape(vector[idx:idx + w1_size], [INPUT_SIZE, HIDDEN_SIZE])
    idx += w1_size
    
    b1 = vector[idx:idx + b1_size]
    idx += b1_size
    
    w2 = tf.reshape(vector[idx:idx + w2_size], [HIDDEN_SIZE, OUTPUT_SIZE])
    idx += w2_size
    
    b2 = vector[idx:idx + b2_size]
    
    return w1, b1, w2, b2


def forward_pass(X, vector):
    w1, b1, w2, b2 = decode_weights(vector)
    
    hidden = sigmoid(tf.matmul(X, w1) + b1)
    output = softmax(tf.matmul(hidden, w2) + b2)
    
    return output

def fitness(vector, X, y_true, batch_size=1000):
    
    indices = np.random.choice(X.shape[0], batch_size, replace=False)
    X_batch = tf.gather(X, indices)
    y_batch = tf.gather(y_true, indices)
    
    y_pred = forward_pass(X_batch, vector)
    loss = tf.keras.losses.categorical_crossentropy(y_batch, y_pred)
    return tf.reduce_mean(loss)
    
# def fitness(vector, X, y_true):
#     y_pred = forward_pass(X, vector)
    
#     loss = tf.keras.losses.categorical_crossentropy(y_true, y_pred)
    
#     return tf.reduce_mean(loss)



In [5]:
total_weights = INPUT_SIZE * HIDDEN_SIZE + HIDDEN_SIZE + HIDDEN_SIZE * OUTPUT_SIZE + OUTPUT_SIZE
vector_np = np.random.randn(total_weights) * 0.01
vector_tf = tf.convert_to_tensor(vector_np, dtype=tf.float32)

loss = fitness(vector_tf, x_train, y_train_hot)
print(f"Initial loss: {loss.numpy()}")


Initial loss: 2.3015012741088867


# functions for the DE

In [6]:


def initialize_population(population_size, D, input_dim=784, output_dim=10):
    
    xavier_scaling = tf.sqrt(6.0 / tf.cast(input_dim + output_dim, tf.float32))
    population = tf.random.uniform(
        shape=(population_size, D),
        minval=-xavier_scaling,
        maxval=xavier_scaling,
        dtype=tf.float32
    )
    
    return population


    


In [7]:

def random_vectors(population, current_idx):
    indices = tf.range(NP)
    mask = indices != current_idx
    valid_indices = tf.boolean_mask(indices, mask)
    selected_indices = tf.random.shuffle(valid_indices)[:3]
    return [population[idx] for idx in selected_indices]

    

In [8]:
def mutate(x1, x2, x3, F, D, L=-1, H=1):

    v = x3 + F * (x1 - x2)
    
    out_of_bounds = tf.logical_or(v < L, v > H) # returns tensor fo booleans where true is erorr
    
    random_values = L + tf.random.uniform(shape=(D,), dtype=tf.float32) * (H - L)
    
    v = tf.where(out_of_bounds, random_values, v) # checks where true and takes value from the random value if false takes from v
    
    return v

 <p>example of how we did the mutate and how mask works</p>
<img src="Screenshot_20250424_015153.png" alt="Project Screenshot" width="600"/>

In [9]:
def crossover(target_vector, mutant_vector, CR, D):

    # instead of loop and check we make the whole vecotor once and check with mask
    r = tf.random.uniform(shape=(D,), dtype=tf.float32)
    
    
    mask = r < CR  # mask is tensor flow boolean vecotr
    
    trial_vector = tf.where(mask, mutant_vector, target_vector)
    
    return trial_vector

# main evolve function

In [10]:

def evolve(population, GEN, NP, D, F, CR, x_train, y_train, fitness_func, L=-0.5, H=0.5):

    # best solution
    fitness_scores = [fitness_func(ind, x_train, y_train) for ind in population]
    best_idx = tf.argmin(fitness_scores)
    best_solution = tf.identity(population[best_idx])  # tf.identity is copy to set it as the best solution 
    best_fitness = fitness_scores[best_idx]
    
    fitness_history = []

    for g in range(GEN):
                
        new_population = [tf.identity(ind) for ind in population] #copy of the population for this generation

        new_fitness_scores = list(fitness_scores) 
        
        for j in range(NP):
            
            v1, v2, v3 = random_vectors(population, j)
            mutant_vector = mutate(v1, v2, v3, F, D, L, H)
            
            target_vector = tf.identity(population[j])         
            trial_vector = crossover(target_vector, mutant_vector, CR, D)
            
            trial_fitness = fitness_func(trial_vector, x_train, y_train)
            target_fitness = new_fitness_scores[j]
            
            if trial_fitness < target_fitness:
                new_population[j] = trial_vector
                new_fitness_scores[j] = trial_fitness
                    
                
                if trial_fitness < best_fitness:
                    best_solution = tf.identity(trial_vector)
                    best_fitness = trial_fitness

        
        
        # each genration swap worst for our best soultion   and send it to next population 
        worst_idx = tf.argmax(new_fitness_scores).numpy()  #get highest fitness / worst indfivsual 
        
        if new_fitness_scores[worst_idx] > best_fitness:
            new_population[worst_idx] = tf.identity(best_solution)
            new_fitness_scores[worst_idx] = best_fitness
            
            
        # upate population with new poulation for next genration 
        for i in range(NP):
            population[i].assign(new_population[i])
        fitness_scores = new_fitness_scores
        
        fitness_history.append(best_fitness)
        
        print(f"Generation {g+1}/{GEN}: Best fitness (loss) = {best_fitness:.6f}")
    
    return best_solution ,fitness_history

In [ ]:
NP = 50
GEN=700
seed = 24
np.random.seed(seed)
tf.random.set_seed(seed)


population = initialize_population(population_size=NP ,D=D)
print("population shape is : ",population.shape)
population = tf.Variable(population)
result, history = evolve(population, GEN, NP, D, 0.8, 0.7, x_train, y_train_hot,fitness, L=-1, H=1)





population shape is :  (50, 25450)
Generation 1/700: Best fitness (loss) = 2.294157
Generation 2/700: Best fitness (loss) = 2.294157
Generation 3/700: Best fitness (loss) = 2.294157
Generation 4/700: Best fitness (loss) = 2.294157
Generation 5/700: Best fitness (loss) = 2.287879
Generation 6/700: Best fitness (loss) = 2.279685
Generation 7/700: Best fitness (loss) = 2.276908
Generation 8/700: Best fitness (loss) = 2.273765
Generation 9/700: Best fitness (loss) = 2.273765
Generation 10/700: Best fitness (loss) = 2.255981
Generation 11/700: Best fitness (loss) = 2.227511
Generation 12/700: Best fitness (loss) = 2.227511
Generation 13/700: Best fitness (loss) = 2.227511
Generation 14/700: Best fitness (loss) = 2.200357
Generation 15/700: Best fitness (loss) = 2.200357
Generation 16/700: Best fitness (loss) = 2.168295
Generation 17/700: Best fitness (loss) = 2.168295
Generation 18/700: Best fitness (loss) = 2.168295
Generation 19/700: Best fitness (loss) = 2.168295
Generation 20/700: Best 

In [ ]:
import plotly.graph_objects as go
fitness_values = [float(tensor.numpy()) for tensor in history]
generations = list(range(len(fitness_values)))

fig = go.Figure()

# Add the fitness trace
fig.add_trace(go.Scatter(
    x=generations,
    y=fitness_values,
    mode='lines',
    name='Fitness (Loss)',
    line=dict(color='blue', width=2)
))


significant_points = []
if len(fitness_values) > 1:
    prev_value = fitness_values[0]
    for i, value in enumerate(fitness_values[1:], 1):
        if value < prev_value - 0.01:  
            significant_points.append((i, value))
            prev_value = value

# Add markers at those points
if significant_points:
    fig.add_trace(go.Scatter(
        x=[p[0] for p in significant_points],
        y=[p[1] for p in significant_points],
        mode='markers',
        marker=dict(color='red', size=8, symbol='circle'),
        name='Significant Improvements'
    ))

fig.update_layout(
    title='Fitness Progress Over Generations',
    xaxis=dict(
        title='Generation',
        tickmode='linear',
        tick0=0,
        dtick=max(1, len(generations) // 20)  
    ),
    yaxis=dict(
        title='Fitness (Loss)',
        range=[max(0, min(fitness_values) - 0.1), min(max(fitness_values) + 0.1, 3.0)]
    ),
    hovermode='x unified',
    template='plotly_white',
    legend=dict(x=0.01, y=0.99, bgcolor='rgba(255,255,255,0.8)'),
    margin=dict(l=20, r=20, t=60, b=20),
    width=900,
    height=500
)

plateau_start = None
for i in range(len(fitness_values) - 20): 
    window = fitness_values[i:i+20]
    if max(window) - min(window) < 0.001:  
        plateau_start = i
        break

if plateau_start is not None:
    fig.add_shape(
        type="line",
        x0=plateau_start, y0=min(fitness_values) - 0.1,
        x1=plateau_start, y1=max(fitness_values) + 0.1,
        line=dict(color="green", width=2, dash="dash"),
    )
    fig.add_annotation(
        x=plateau_start, y=fitness_values[plateau_start],
        text="Convergence plateau begins",
        showarrow=True,
        arrowhead=1,
    )

fig.show()

# Print some statistics
print(f"Starting fitness: {fitness_values[0]:.6f}")
print(f"Final fitness: {fitness_values[-1]:.6f}")
print(f"Improvement: {fitness_values[0] - fitness_values[-1]:.6f} ({(fitness_values[0] - fitness_values[-1])/fitness_values[0]*100:.2f}%)")
print(f"Number of generations: {len(fitness_values)}")




In [ ]:
def evaluate_model(best_solution, x_test, y_test):

    # Get predictions
    probabilities = forward_pass(x_test, best_solution)
    predictions = tf.argmax(probabilities, axis=1)
    true_labels = tf.argmax(y_test, axis=1)
    
    # Calculate accuracy
    correct = tf.equal(predictions, true_labels)
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))
    
    # Calculate loss
    loss = tf.reduce_mean(tf.keras.losses.categorical_crossentropy(y_test, probabilities))
    
    print(f"Test accuracy: {accuracy.numpy() * 100:.2f}%")
    print(f"Test loss: {loss.numpy():.4f}")
    
    return accuracy.numpy(), predictions.numpy(), probabilities.numpy()

In [ ]:
acc, pred , prob = evaluate_model(result, x_test,y_test_hot)

In [ ]:
def visualize_predictions(x_test, y_test, predictions, probabilities, num_samples=5):

    import matplotlib.pyplot as plt
    import numpy as np
    import random
    
    # Get random samples
    indices = random.sample(range(len(x_test)), num_samples)
    
    # Set up the figure
    fig, axes = plt.subplots(num_samples, 2, figsize=(12, 2*num_samples))
    
    for i, idx in enumerate(indices):
        # Original image
        original_image = x_test[idx].numpy().reshape(28, 28)
        true_label = np.argmax(y_test[idx])
        pred_label = predictions[idx]
        pred_prob = probabilities[idx][pred_label]
        
        # Show image
        axes[i, 0].imshow(original_image, cmap='gray')
        axes[i, 0].set_title(f"True: {true_label}")
        axes[i, 0].axis('off')
        
        # Show probabilities
        axes[i, 1].bar(range(10), probabilities[idx])
        axes[i, 1].set_title(f"Pred: {pred_label} (Confidence: {pred_prob:.4f})")
        axes[i, 1].set_xticks(range(10))
        
    plt.tight_layout()
    plt.show()

In [ ]:
visualize_predictions(x_test, y_test_hot,pred,prob,10)